In [4]:
#import sagemaker
#import boto3
#from sagemaker.amazon.amazon_estimator import get_image_uri 
#from sagemaker.session import s3_input, Session

In [5]:
#bucket_name = 'nlp-spam' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
#my_region = boto3.session.Session().region_name # set the region of the instance
#print(my_region)


ap-south-1


In [1]:
'''
s3=boto3.resource('s3')
try: 
    if my_region=='us-east-2':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
    print('done bucket has been created')
except Exception as e:
    print('exception: ',e)
    
'''

"\ns3=boto3.resource('s3')\ntry: \n    if my_region=='us-east-2':\n        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})\n    print('done bucket has been created')\nexcept Exception as e:\n    print('exception: ',e)\n    \n"

In [3]:
'''prefix = 'spam-classification'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)
'''

NameError: name 'bucket_name' is not defined

In [2]:
import pandas as pd

In [3]:
data=pd.read_csv('SMSSpamCollection',sep='\t',names=["labels","messages"])

In [4]:
data.head()

,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
#nltk.download('all')
import re
!pip install gensim
from gensim.models import Word2Vec

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [6]:
stemmer = PorterStemmer()
lem=WordNetLemmatizer()
corpus=[]

In [7]:
for i in range(0,len(data)):
    review=re.sub('[^a-zA-z]',' ',data['messages'][i])
    review=review.lower()
    review=review.split()
    words = [stemmer.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(words)   
    corpus.append(review)

In [9]:
len(corpus)

5572

In [17]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer(max_features=2500)
#cv1=TfidfVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

y=pd.get_dummies(data['labels'])
y=y.iloc[:,1].values

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [19]:

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [21]:
accuracy_score(y_test,y_pred)

0.9847533632286996

In [22]:
confusion_matrix(y_test,y_pred)

array([[946,   9],
       [  8, 152]])